In [71]:
import numpy as np
import pandas as pd


In [72]:
cluster_fp = "Data\\fine_clusters_birch_thresh_60_with_colornames.csv"
df_clusters = pd.read_csv(cluster_fp, delimiter=";")
df_clusters.head()

,Unnamed: 0.1,Unnamed: 0,id,platform,hashtag,r,g,b,h,s,v,relative_frequency,colorname
0,0,0,7172368921318657322,tiktok,climatechange,0.892157,0.816215,0.676990,0.107843,0.241176,0.892157,0.004774,yellow
1,1,0,7172368921318657322,tiktok,climatechange,0.678431,0.416637,0.242107,0.066667,0.643137,0.678431,0.086914,orange-yellow
2,2,0,7172368921318657322,tiktok,climatechange,0.531373,0.531373,0.531373,0.000000,0.000000,0.531373,0.031250,gray
3,3,0,7172368921318657322,tiktok,climatechange,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.099501,black
4,4,0,7172368921318657322,tiktok,climatechange,0.482353,0.272388,0.321791,0.960784,0.435294,0.482353,0.072374,red-orange


In [73]:
columns = ["id", "platform", "hashtag", "n_clusters",
                                  "mean_r", "mean_g", "mean_b", "mean_h", "mean_s", "mean_v",
                                  "dominant_r", "dominant_g", "dominant_b", "dominant_h", "dominant_s", "dominant_v",
                                  "black", "gray", "white", "orange", "orange-yellow", "yellow", "yellow-green", "green", "green-blue", "blue", "blue-violet", "violet", "violet-red", "red", "red-orange"]
df_images = pd.DataFrame(columns=columns)

In [74]:
total_n_images = len(df_clusters["id"].unique())
n_images = 0
for image_id in df_clusters["id"].unique():
    image_specific_clusters = df_clusters[df_clusters["id"] == image_id]
    n_clusters = len(image_specific_clusters)
    platform = image_specific_clusters.platform.mode()[0]
    hashtag = image_specific_clusters.hashtag.mode()[0]
    init_values = [[image_id, 
                    platform,
                    hashtag,
                    n_clusters,
                    0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                    0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
                    0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
    new_row = pd.DataFrame(init_values,columns=columns)
    new_row.at[0,"id"] = image_id
    #new_row.at[0,"platform"] = platform
    #new_row.at[0,"hashtag"] = hashtag
    new_row.at[0,"n_clusters"] = n_clusters

    max_frac = -1.0
    for idx, row in image_specific_clusters.iterrows():
        frac = row["relative_frequency"]
        if frac > max_frac:
            max_frac = frac
            new_row.at[0,"dominant_h"] = row["h"]
            new_row.at[0,"dominant_s"] = row["s"]
            new_row.at[0,"dominant_v"] = row["v"]
            new_row.at[0,"dominant_r"] = row["r"]
            new_row.at[0,"dominant_g"] = row["g"]
            new_row.at[0,"dominant_b"] = row["b"]

        new_row.at[0,"mean_h"] += frac * row["h"]
        new_row.at[0,"mean_s"] += frac * row["s"]
        new_row.at[0,"mean_v"] += frac * row["v"]
        new_row.at[0,"mean_r"] += frac * row["r"]
        new_row.at[0,"mean_g"] += frac * row["g"]
        new_row.at[0,"mean_b"] += frac * row["b"]

        new_row.at[0,row["colorname"]] += frac
    
    print (f"\r {n_images}/{total_n_images}", end="")
    n_images += 1
    #print(image_id)
    #print(new_row)
    
    df_images = pd.concat([df_images, new_row], axis=0)
    #print(df_images["platform"])
df_images.head()

 5440/5441

,id,platform,hashtag,n_clusters,mean_r,mean_g,mean_b,mean_h,mean_s,mean_v,...,yellow,yellow-green,green,green-blue,blue,blue-violet,violet,violet-red,red,red-orange
0,7172368921318657322,tiktok,climatechange,12,0.229980,0.304649,0.391277,0.512652,0.561194,0.447482,...,0.004774,0.0,0.0,0.000000,0.0,0.169162,0.035156,0.131619,0.000000,0.072374
0,7256810993311812907,tiktok,climatechange,14,0.539718,0.498850,0.517335,0.379549,0.245650,0.607623,...,0.000000,0.0,0.0,0.000000,0.0,0.115126,0.112522,0.000000,0.000000,0.000000
0,7224978138751618305,tiktok,climatechange,12,0.251142,0.194146,0.203548,0.451293,0.375548,0.275581,...,0.000000,0.0,0.0,0.000000,0.0,0.257161,0.000000,0.000000,0.027886,0.000000
0,7291671021827116334,tiktok,climatechange,15,0.388935,0.235357,0.250895,0.464454,0.522460,0.420612,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.130751,0.000000,0.000000,0.000000
0,7252904683222895918,tiktok,climatechange,12,0.519744,0.507381,0.439802,0.319209,0.269828,0.562161,...,0.230794,0.0,0.0,0.081597,0.0,0.000000,0.041667,0.000000,0.000000,0.029514


In [78]:
fp_output = "Data\\unified_color_data_birch.csv"
df_images.to_csv(fp_output)